# Purpose:  
## Will make commercial proposal Using data about   reviews of drugs.
 
 # Tasks:
  1. Preprocessing data(drop dublicatee, punctuation marks)
  2. Train the model for predict rating of review   
  4. Explore data for make commercial proposal

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import unicodedata
import re
import numpy as np
from multiprocessing import Pool
import sys
import plotly.offline as py
import plotly.graph_objs as go
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     D:\anaconda\envs\datasience\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     D:\anaconda\envs\datasience\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     D:\anaconda\envs\datasience\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     D:\anaconda\envs\datasience\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# We will using only this file drugsComTrain_raw.tsv
# If you want, you cne use test file for testing model
train_data = pd.read_csv("drugsComTrain_raw.tsv", sep='\t')

In [3]:
train_data = train_data.drop(columns = ['Unnamed: 0', 'date'])

### There are identical reviews for different medicines, since we will predict the drug's rating based on the comment, and we will remove duplicates

In [4]:
# Drop dublicete of review
train_data.drop(train_data.review[~train_data.review.duplicated()].index, inplace=True)

In [5]:
train_data

,drugName,condition,review,rating,usefulCount
524,Nexplanon,Birth Control,"""First had implanon then got Nexplanon, had a ...",9.0,5
574,Cymbalta,Anxiety,"""Prescribed via a Psychiatrist for severe Pani...",1.0,27
726,Orsythia,Birth Control,"""I have only been on orsythia for about 1 mont...",2.0,7
1070,Desvenlafaxine,Depression,"""I have suffered from severe anxiety (GAD) and...",8.0,81
1375,Ethinyl estradiol / norethindrone,Birth Control,"""I have been taking my first pack of Lo Loestr...",8.0,7
...,...,...,...,...,...
161285,Alcaftadine,"Conjunctivitis, Allergic","""Used this a month ago for allergic conjunctiv...",1.0,12
161288,Fluoxetine,Obsessive Compulsive Disorde,"""I have been off Prozac for about 4 weeks now....",8.0,22
161291,Junel 1.5 / 30,Birth Control,"""This would be my second month on Junel. I&#03...",6.0,0
161293,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1.0,34


In [6]:
# Drop numbers in text 
train_data['review']=train_data['review'].map(lambda w: re.sub(r'[^\w\s]+|[\d]+', r'',w).strip())

In [7]:
# Drop all punctuation marks
punct= dict.fromkeys(i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P'))
train_data['review'] = train_data['review'].map(lambda text: text.lower().translate(punct))

In [8]:
# Drop stop words
def drop_sw(text):
    word = word_tokenize(text)
    word = [i for i in word if i not in stop and not i.isdigit()]
    return ' '.join(map(str, word))

train_data['review'] = train_data['review'].map(drop_sw)

In [9]:
train_data.head()

,drugName,condition,review,rating,usefulCount
524,Nexplanon,Birth Control,first implanon got nexplanon period first mont...,9.0,5
574,Cymbalta,Anxiety,prescribed via psychiatrist severe panic attac...,1.0,27
726,Orsythia,Birth Control,orsythia month started second week second mont...,2.0,7
1070,Desvenlafaxine,Depression,suffered severe anxiety gad taking klonopin ti...,8.0,81
1375,Ethinyl estradiol / norethindrone,Birth Control,taking first pack lo loestrin fe must say real...,8.0,7


In [10]:
# This dict will use for detect part of language 
def word_tag(word):
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

Do lemantize for rewiew 


In [11]:
lem = WordNetLemmatizer()

In [12]:
train_data['review'] = train_data['review'].map(lambda w:" ".join([lem.lemmatize(i, word_tag(i)) for i in word_tokenize(w)]))

In [13]:
train_data.head()

,drugName,condition,review,rating,usefulCount
524,Nexplanon,Birth Control,first implanon get nexplanon period first mont...,9.0,5
574,Cymbalta,Anxiety,prescribed via psychiatrist severe panic attac...,1.0,27
726,Orsythia,Birth Control,orsythia month start second week second month ...,2.0,7
1070,Desvenlafaxine,Depression,suffer severe anxiety gad take klonopin time g...,8.0,81
1375,Ethinyl estradiol / norethindrone,Birth Control,take first pack lo loestrin fe must say really...,8.0,7


In [14]:
# I save file for will not repeat preprocesing data, if something goes wrong
train_data.to_csv('tabletki_train.csv')

In [15]:
pd.set_option('display.max_rows', 5000) 

In [16]:
# read file
train_data = pd.read_csv('tabletki_train.csv')

In [17]:
# Convert type
train_data.condition = train_data.condition.astype('str')
train_data.rating=train_data.rating.astype(int)

In [18]:
# This function replce condition wiht on mist common condition for this drugName
def replace_condition(index):
    d_name = train_data.loc[index:index].drugName.values[0]
    most_com = train_data[train_data.drugName==d_name].condition.value_counts().index[0]
    train_data.loc[index:index].condition=most_com 

In [19]:
for i  in train_data[train_data.condition=='nan'].index:
    replace_condition(i)

In [20]:
train_data.isna().sum()

Unnamed: 0     0
drugName       0
condition      0
review         3
rating         0
usefulCount    0
dtype: int64

In [21]:
train_data.dropna(inplace=True)

In [22]:
train_data.isna().sum()

Unnamed: 0     0
drugName       0
condition      0
review         0
rating         0
usefulCount    0
dtype: int64

### Now i want test my hypothesis. If we will train one model for predict 10 classes it gives a so small sсore.### 
Now watch your hands. For example, take a problem where there are 3 classes. We will create a model that will predict whether it is a class 1 based on the fact that it is not 2. If it is exactly the first class, the model will tell us that it is exactly 1 class, if not 1 then 2. So I want to predict 10 classes.And for each class there will be 9 models that will test hypotheses and give the result. I do not know if you understand, if that write to me I will explain to you

In [23]:
# Create dict. In this dict first keys it name of prediction class(1..10), each
# key has dict with keys(1/2, 1/3, 1/4 ....) and values X and y
test_hyp ={}
for i in sorted(train_data.rating.unique()):
  test_hyp[f'{int(i)}']={
      f'{int(i)}/{int(j)}': {'X':train_data['review'][(train_data.rating == i) |
                                                      (train_data.rating==j)],
                             'y':train_data.rating[(train_data.rating == i) | 
                                                   (train_data.rating==j)]}  
      for j in sorted(train_data.rating.unique()) if j!= i
      }

In [24]:
# create obj of TfidfVectorizer() and traing 
tdf_vec = TfidfVectorizer()
tdf_vec.fit(train_data.review)

TfidfVectorizer()

In [25]:
# create obj of TruncatedSVD and traing 
trun_svd = TruncatedSVD(n_components=700)
trun_svd.fit(tdf_vec.transform(train_data.review))

TruncatedSVD(n_components=700)

In [26]:
# create model for all class 
# i didn't split data on test and train, because now i show how it's working, test this solutions we will next
for key_1 in test_hyp.keys():
  for key_2 in test_hyp[key_1].keys():
    test_hyp[key_1][key_2]['X']= trun_svd.transform(
        tdf_vec.transform(test_hyp[key_1][key_2]['X']))
    test_hyp[key_1][key_2]['model'] = LogisticRegression(n_jobs=-1).fit(
         test_hyp[key_1][key_2]['X'], test_hyp[key_1][key_2]['y']
     )

In [27]:
#show how we predict class 
def predict_rating(review):
  review = trun_svd.transform(tdf_vec.transform([review]))
  result = {}
  for key_1 in test_hyp.keys():
    prom_res = []
    for key_2 in test_hyp[key_1].keys():
      prom_res.append(test_hyp[key_1][key_2]['model'].predict(review))
      result[prom_res.count(int(key_1))/len(prom_res)] = key_1

  return result[max(list(result.keys()))]

print(predict_rating('ive chronic constipation adult life try linz work month stop doctor start amitiza mg miracle ive four month life day bowel motion go comfortably time per week pain little gas miracle'))      


  

10


### Now i create class for multiclassing classification. You will can this class for other task


In [28]:
class Predict:

    def __init__(self, data_frame, text_col, class_col):
      # Initialize all necessary variables
        self.data = data_frame
        self.name_text = text_col
        self.name_class = class_col
        self.tdf_vec = TfidfVectorizer()
        self.trun_svd = TruncatedSVD(n_components=700)
        self._train_tdf_trun()
        self.working_dict = self._create_dict()
        self.dict_with_model = self._add_model_to_dict()

    def _train_tdf_trun(self):
      # Create and train  TfidfVectorizer()
        self.tdf_vec.fit(self.data[self.name_text])
        self.trun_svd.fit(self.tdf_vec.transform(self.data[self.name_text]))

    def _create_dict(self):
      # Create dict with values 
        test_hyp = {}
        for i in sorted(self.data.rating.unique()):
            test_hyp[f'{int(i)}'] = {
                f'{int(i)}/{int(j)}': {'X': self.data[self.name_text][(self.data[self.name_class] == i) |
                                                                      (self.data[self.name_class] == j)],
                                       'y': self.data[self.name_class][(self.data[self.name_class] == i)
                                                                       | (self.data[self.name_class] == j)]}
                for j in sorted(self.data[self.name_class].unique()) if j != i
            }
        return test_hyp

    def _add_model_to_dict(self):
      # This function add model in dict 
        test_hyp = self._create_dict()
        for key_1 in test_hyp.keys():
            for key_2 in test_hyp[key_1].keys():
                test_hyp[key_1][key_2]['X'] = self.trun_svd.transform(
                    self.tdf_vec.transform(test_hyp[key_1][key_2]['X']))
                test_hyp[key_1][key_2]['model'] = LogisticRegression(n_jobs=-1).fit(
                    test_hyp[key_1][]hyp[key_1][key_2]['y']
                )

        return test_hyp

    def predict_class(self, review):
      # Predict class
        review = self.trun_svd.transform(self.tdf_vec.transform([review]))
        result = {}
        for key_1 in self.dict_with_model.keys():
            prom_res = []
            for key_2 in self.dict_with_model[key_1].keys():
                prom_res.append(self.dict_with_model[key_1][key_2]['model'].predict(review))
            result[prom_res.count(int(key_1)) / len(prom_res)] = key_1

        return result[max(list(result.keys()))]

    @staticmethod
    def metrics(tru._rez, predict_rez):
      # It is function for get score 
        if abs(true_rez - predict_rez) == 1 or abs(true_rez - predict_rez) == 0:
            return 1
        else:
            return 0


SyntaxError: invalid syntax (<ipython-input-28-bfb531f5cea6>, line 40)

### Next we will testing class

In [37]:
train_class = train_data.sample(frac=0.7, random_state=50) 
test_class = train_data.drop(index=train_class.index , axis=1)

In [38]:
new_obj = Predict(train_class, 'review', 'rating')

In [39]:
review = test_class.review.values
true_rating = test_class.rating.values

In [40]:
resul_met = [new_obj.metrics(int(new_obj.predict_class(rev)), rat) for rev, rat in zip(review, true_rating)]
print(resul_met.count(1)/len(resul_met))

0.6182177139355981


In [48]:
log_reg = LogisticRegression(n_jobs=-1)
tr_f = trun_svd.transform(tdf_vec.transform(train_class.review))
test_f = trun_svd.transform(tdf_vec.transform(review))
log_reg.fit(tr_f, train_class.rating)
print(log_reg.score(test_f, true_rating))

0.4025461229491456


In [ ]:
# def finde_param(n):
#   trun_svd = TruncatedSVD(n_components=n)
#   trun_svd.fit(tdf_vec.transform(small_data.review))
#   for key_1 in test_hyp.keys():
#     for key_2 in test_hyp[key_1].keys():
#       test_hyp[key_1][key_2]['X']= trun_svd.transform(
#           tdf_vec.transform(test_hyp[key_1][key_2]['X']))
#       X_train, X_test, y_train, y_test = train_test_split(
#           test_hyp[key_1][key_2]['X'], 
#           test_hyp[key_1][key_2]['y'], test_size = 0.3,  random_state = 7)
#       test_hyp[key_1][key_2]['model'] = LogisticRegression(n_jobs=-1).fit(
#           X_train, y_train
#       ) 
#       test_hyp[key_1][key_2]['score'] = test_hyp[key_1][key_2]['model'].score(
#           X_test, y_test
#       )

### Since these data are reviews of medicines, we will try to conduct a small marketing research

In [31]:
explore_data = pd.read_csv('tabletki_train.csv')
explore_data.drop('Unnamed: 0', axis=1, inplace=True)
explore_data.head()

,drugName,condition,review,rating,usefulCount
0,Nexplanon,Birth Control,first implanon get nexplanon period first mont...,9.0,5
1,Cymbalta,Anxiety,prescribed via psychiatrist severe panic attac...,1.0,27
2,Orsythia,Birth Control,orsythia month start second week second month ...,2.0,7
3,Desvenlafaxine,Depression,suffer severe anxiety gad take klonopin time g...,8.0,81
4,Ethinyl estradiol / norethindrone,Birth Control,take first pack lo loestrin fe must say really...,8.0,7


In [32]:
df_forplt=explore_data.groupby('condition').sum('usefulCount').sort_values('usefulCount', ascending=False)

In [33]:
df_forplt.head()

,rating,usefulCount
condition,,
Depression,19705.0,153305
Anxiety,12380.0,93447
Birth Control,64949.0,82761
Pain,13290.0,67325
Bipolar Disorde,9581.0,53572


In [34]:
gist_1 = go.Bar(y=[i for i in df_forplt.usefulCount.values[:10]], 
                   x=[i for i in df_forplt.index.values[:10]]
            , marker=dict(color = 'rgb(200, 1, 174)'))
go.Figure(data=[gist_1]).update_layout(title="How many people rated")

### Based on this graph, we can see that people with depression often pay more attention to text comments and find them useful. That contextual advertising of anti-depression drugs can boost sales and that people with depression are more likely to listen to other people's opinions. Next we will look at the words used in the comments for drugs against depression


In [35]:
explore_data[(explore_data.condition=='Depression') & 
             (explore_data.usefulCount>50) & 
             (explore_data.rating>6)].sort_values('usefulCount', ascending=False).head(10)

,drugName,condition,review,rating,usefulCount
2173,Zoloft,Depression,remember reading people opinion online drug ta...,10.0,1291
43995,Celexa,Depression,respond one week side effect lethargy bowel pr...,8.0,771
15393,Vilazodone,Depression,lot additional stress add last year death pare...,10.0,693
38829,Desvenlafaxine,Depression,try several top ten antidepressant never feel ...,10.0,504
31479,Escitalopram,Depression,wasnt severely depressed always crazy sensitiv...,9.0,458
5652,Duloxetine,Depression,cymbalta best thing ever try depression also s...,10.0,446
23926,Sertraline,Depression,im write post spent lot time research online z...,8.0,426
13933,Pristiq,Depression,suffer dysthymic depression prozac stop work s...,10.0,389
36034,Lexapro,Depression,ive lexapro several year say initial side effe...,8.0,376
47359,Vilazodone,Depression,finish th week viibryd im afraid jinx say feel...,9.0,366


In [36]:
dep_word = " ".join(list(explore_data.review.astype(str).values))

In [37]:
dep_word = word_tokenize(dep_word)

In [38]:
most_com=Counter(dep_word).most_common(10)

In [39]:
gist_2 = go.Bar(y=[i[1] for i in most_com], x=[i[0] for i in most_com], 
                marker=dict(color = 'rgb(200, 1, 174)'))
go.Figure(data=[gist_2]).update_layout(title="Most common word in review adout depresion")

### In the graph, we can see the 10 most common words in the comments of anti-depression medications. But not only the words are important to us, but also in what context they are used### 
Then we will display the comment that users found most useful


### This 3 review with the bigest usefulcout.
    
1. remember reading people opinion online drug take scar away finally decide give try best choice make month feel great im mg dont side effect first start notice hand would tremble subside honestly dont listen negativity doesnt work work amaze others go base youself everyone else may blessing diguise pill meant make happy go lucky see quotbutterflies rosesquot meant help put chemical mind balance overly depressed still get sad time normal life people take control make change get pill    
2.ive zoloft mg two year relaxed easygoing person zoloft prior start medicine anxious time worried constantly everything suffer period depression hopelessness well wife friend person genera    l
3.'respond one week side effect lethargy bowel problem reason didnt rate medicine year excessive anxiety depression finally feel like quotmyselfquot instead enslave unreasonable thought frustration ironically avoid type med year instead quotself medicatingquot alcohol cynicism switch year ago maybe would still married wonderful woman sad true### '

Using them and frequently encountered words, we can generate texts that will have a greater impact on the choice of a particular lecture.

 # Commercial offer: 
 ### Using this data, it is possible to generate reviews for medicines, which can be used by bots to write them on forums to create a good rating of medicines. But now I can't implement it.

# Conclusion:
### In the process of completing the work, we consolidated our skills in working with text, wrote our own classification algorithm and method for evaluating the result. Made a commercial offer

